In [16]:
import pickle
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from biosppy.signals.tools import filter_signal
from biosppy.signals import ecg
import json

In [13]:
# def filter_signals(sig, fs):
#     sig_filtered = [filter_signal(sig[:,0],
#                     ftype='FIR',
#                     band='bandpass',
#                     order=50,
#                     frequency=[0.5,45],
#                     sampling_rate=fs)[0],
#                     filter_signal(sig[:,1],
#                     ftype='FIR',
#                     band='bandpass',
#                     order=50,
#                     frequency=[0.5,45],
#                     sampling_rate=fs)[0]
#                     ]
#     return sig_filtered

# def get_rpeaks(sig_filtered, fs):
#     rpeaks = ecg.christov_segmenter(sig_filtered[0], fs)[0]
#     return rpeaks.tolist()

# def preprocessing(data):
#     fs = data['fs']
#     # filter data
#     sig = filter_signals(data['sig'], fs)
        
#     # get R-peaks
#     rpeaks = get_rpeaks(sig, fs)
        
#     sig = np.transpose(sig)
        
#     return sig.tolist(), rpeaks

def get_train(train):
    new_train = []
    for i in range(len(train)):
        print(str(i+1) + '/' + str(len(train)), end='\r')
        # filter signals and get rpeaks
#         train[i]['sig_filtered'] = [filter_signal(train[i]['sig'][:,0],
#                                     ftype='FIR',
#                                     band='bandpass',
#                                     order=50,
#                                     frequency=[0.5,45],
#                                     sampling_rate=train[i]['fs'])[0],
#                                     filter_signal(train[i]['sig'][:,1],
#                                     ftype='FIR',
#                                     band='bandpass',
#                                     order=50,
#                                     frequency=[0.5,45],
#                                     sampling_rate=train[i]['fs'])[0]
#                                     ]
#         train[i]['rpeaks'] = ecg.christov_segmenter(train[i]['sig_filtered'][0], train[i]['fs'])[0].tolist()
#         train[i]['sig'] = train[i]['sig'].tolist()
#         train[i]['beat_loc'] = train[i]['beat_loc'].tolist()
#         train[i]['af_start_scripts'] = train[i]['af_start_scripts'].tolist()
#         train[i]['af_end_scripts'] = train[i]['af_end_scripts'].tolist()

        tmp = dict()
        tmp['record_name'] = train[i]['record_name']
        tmp['fs'] = train[i]['fs']
        tmp['sig_filtered'] = np.transpose([filter_signal(train[i]['sig'][:,0],
                                               ftype='FIR',
                                               band='bandpass',
                                               order=50,
                                               frequency=[0.5,45],
                                               sampling_rate=train[i]['fs'])[0],
                                               filter_signal(train[i]['sig'][:,1],
                                               ftype='FIR',
                                               band='bandpass',
                                               order=50,
                                               frequency=[0.5,45],
                                               sampling_rate=train[i]['fs'])[0]
                                               ])
        tmp['rpeaks'] = ecg.christov_segmenter(tmp['sig_filtered'][:,0], train[i]['fs'])[0].tolist()
        tmp['sig_filtered'] = tmp['sig_filtered'].tolist()
        tmp['af_start'] = [train[i]['beat_loc'][x] for x in train[i]['af_start_scripts']]
        tmp['af_end'] = [train[i]['beat_loc'][x] for x in train[i]['af_end_scripts']]
        tmp['class_true'] = train[i]['class_true']
        
        new_train.append(tmp)
        break

    return new_train

In [5]:
TRAINING_DATA_PATH = '../data/train.pkl'
with open(TRAINING_DATA_PATH, 'rb') as file:
    train = pickle.load(file)

In [6]:
train[0].keys()

dict_keys(['record_name', 'sig', 'sig_len', 'fs', 'beat_loc', 'af_start_scripts', 'af_end_scripts', 'class_true'])

In [14]:
new_train = get_train(train)

In [17]:
with open('../data/train.json', 'w') as file:
    json.dump(new_train, file)